# Prueba de aprendizaje no supervisado de detección de filtros.



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Configuracion de carpetas
carpeta_drive_base = "drive/My Drive/Colab Notebooks/"
carpeta_drive_prueba_red = "drive/My Drive/Colab Notebooks/Prueba red/"
carpeta_drive_cifar = "drive/My Drive/Colab Notebooks/Prueba red/cifar_100_python/"

In [0]:
# Importaciones
from __future__ import print_function

import os
import sys

import tensorflow 
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.datasets import cifar10, mnist


from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
import cv2
import numpy as np
from matplotlib import pyplot as plt
import importlib.util
from keras.utils import to_categorical

In [0]:
# Uso de imagenes de cifar
# (X_train, _), (X_test, _) = cifar10.load_data()
(X_train, _), (X_test, _) = mnist.load_data()


11493376/11490434 [==============================] - 1s 0us/step


In [0]:
imagenes_filtradas = []
imagenes_filtradas_test = []

#utilización de 10000 imagenes, todas
X_train = X_train[:]

In [0]:
# Filtro de promediado
kernel = np.ones((5,5),np.float32)/25
for img in X_train:
  #Filtra la imagen utilizando el kernel anterior
  filt = cv2.filter2D(img,-1,kernel)
  imagenes_filtradas.append(filt)

for img in X_test:
  #Filtra la imagen utilizando el kernel anterior
  filt = cv2.filter2D(img,-1,kernel)
  imagenes_filtradas_test.append(filt)

In [0]:
# Filtro Difuminado 
for img in X_train:
  filt = cv2.blur(img,(3,3))
  imagenes_filtradas.append(filt)

for img in X_test:
  filt = cv2.blur(img,(3,3))
  imagenes_filtradas_test.append(filt)

In [0]:
# Filtro gaussiano
for img in X_train:
  filt = cv2.GaussianBlur(img,(5,5),0)
  imagenes_filtradas.append(filt)

for img in X_test:
  filt = cv2.GaussianBlur(img,(5,5),0)
  imagenes_filtradas_test.append(filt)

In [0]:
# Filtro mediano
for img in X_train:
  filt = cv2.medianBlur(img,5)
  imagenes_filtradas.append(filt)

for img in X_test:
  filt = cv2.medianBlur(img,5)
  imagenes_filtradas_test.append(filt)

In [0]:
# Objetivo
y_train = [1 for i in range(len(X_train))] + [2 for i in range(len(X_train))] + [3 for i in range(len(X_train))] + [4 for i in range(len(X_train))]
y_train = to_categorical(y_train)

y_test = [1 for i in range(len(X_test))] + [2 for i in range(len(X_test))] + [3 for i in range(len(X_test))] + [4 for i in range(len(X_test))]
y_test = to_categorical(y_test)

array([0., 0., 1., 0., 0.], dtype=float32)

In [0]:
# Configuracion 
epocas=50
longitud, altura = 28, 28
batch_size = 250
pasos = 1000
validation_steps = 300
clases = 4
lr = 0.0004


In [0]:
imagenes_filtradas = np.array(imagenes_filtradas)
y_train = np.array(y_train)

imagenes_filtradas_test = np.array(imagenes_filtradas_test)
y_test = np.array(y_test)

In [0]:
spec = importlib.util.spec_from_file_location("modelo_resnet3", "drive/My Drive/Colab Notebooks/Prueba red/modelo_resnet3.py")
archivo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(archivo)


model = archivo.ResnetBuilder.build_resnet_18((1,28, 28), 5)

dont tf


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
imagenes_filtradas = imagenes_filtradas.reshape(imagenes_filtradas.shape[0],28,28,1)
imagenes_filtradas_test = imagenes_filtradas_test.reshape(imagenes_filtradas_test.shape[0],28,28,1)

60000

In [0]:
# Train con validación
model.fit(
        x=imagenes_filtradas,y=y_train,
        batch_size = 320,
        epochs=epocas,
        validation_data=[imagenes_filtradas_test,y_test])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 40000 samples
Epoch 1/50
60000/60000 [==============================] - 27s 443us/step - loss: 0.6171 - acc: 0.9793 - val_loss: 1.4432 - val_acc: 0.6983
Epoch 2/50
60000/60000 [==============================] - 13s 210us/step - loss: 0.3788 - acc: 0.9945 - val_loss: 0.5755 - val_acc: 0.8947
Epoch 3/50
60000/60000 [==============================] - 13s 212us/step - loss: 0.2590 - acc: 0.9991 - val_loss: 0.2335 - val_acc: 0.9982
Epoch 4/50
60000/60000 [==============================] - 13s 215us/step - loss: 0.1876 - acc: 0.9989 - val_loss: 0.3793 - val_acc: 0.9023
Epoch 5/50
60000/60000 [==============================] - 13s 219us/step - loss: 0.1428 - acc: 0.9982 - val_loss: 0.1219 - val_acc: 0.9994
Epoch 6/50
60000/60000 [==============================] - 13s 219us/step - loss: 0.1188 - acc: 0.9969 - val_loss: 0.1064 - val_acc: 0.9976
Epoch 7/50
60000/600

In [0]:
# Train sin validación
model.fit(
        x=imagenes_filtradas,y=y_train,
        batch_size = 320,
        epochs=epocas)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
40000/40000 [==============================] - 33s 834us/step - loss: 1.7478 - acc: 0.4772
Epoch 2/30
40000/40000 [==============================] - 22s 538us/step - loss: 1.1094 - acc: 0.7188
Epoch 3/30
40000/40000 [==============================] - 22s 538us/step - loss: 0.8769 - acc: 0.7836
Epoch 4/30
40000/40000 [==============================] - 21s 533us/step - loss: 0.7409 - acc: 0.8167
Epoch 5/30
40000/40000 [==============================] - 21s 529us/step - loss: 0.6757 - acc: 0.8268
Epoch 6/30
40000/40000 [==============================] - 21s 529us/step - loss: 0.6193 - acc: 0.8383
Epoch 7/30
40000/40000 [==============================] - 21s 529us/step - loss: 0.5622 - acc: 0.8502
Epoch 8/30
40000/40000 [==============================] - 21s 529us/step - loss: 0.5245 - acc: 0.8609
Epoch 9/30
40000/40000 [==============================] - 21s 529us/step - loss: 0.5231 

In [0]:
##
#Arquitectura del modelo base
from __future__ import division

import six
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
    concatenate
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K


def _bn_relu(input):
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)


def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f


def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f


def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks.
    """
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
            print(input)
        return input

    return f


def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                              strides=init_strides,
                              padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)

    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    
    if K.image_data_format() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
        print("yes tf")
    else:
        print("dont tf")
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        # if K.image_data_format() == 'tf':
        input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        cont_block = 0
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)

            filters *= 2

        # Last activation
        block = _bn_relu(block)
    
        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)
        flatten1 = Flatten()(pool2)
        input2 = Input(shape=(1,512))
        
        new_dens = Dense(512, activation="relu")(input2)
        flatten2 = Flatten()(new_dens)
        combined = concatenate([flatten1, flatten2])
        pre = Dense(512, activation="relu")(combined)
        
        dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      activation="softmax")(pre)

        model = Model(inputs=[input, input2], outputs=dense)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])

    @staticmethod
    def build_resnet_152(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])

In [0]:
modelo_resnet = ResnetBuilder.build_resnet_18((1,28, 28), 10)
modelo_resnet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

dont tf
Tensor("add_17/add:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("add_18/add:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("add_19/add:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("add_20/add:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("add_21/add:0", shape=(?, 2, 2, 256), dtype=float32)
Tensor("add_22/add:0", shape=(?, 2, 2, 256), dtype=float32)
Tensor("add_23/add:0", shape=(?, 1, 1, 512), dtype=float32)
Tensor("add_24/add:0", shape=(?, 1, 1, 512), dtype=float32)



In [0]:
# serialize model to JSON
model_json = model.to_json()
with open(carpeta_drive_prueba_red + "model_selfsupervised_filter_mnist.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(carpeta_drive_prueba_red + "model_selfsupervised_filter_mnist.h5")
print("Saved model to disk")

Saved model to disk


# Modelo en conjunto filter

In [0]:
#Cargando modelo de filtros
from keras.models import model_from_json

json_file = open(carpeta_drive_prueba_red + 'model_selfsupervised_filter_mnist.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_filter = model_from_json(loaded_model_json)
# load weights into new model
model_filter.load_weights(carpeta_drive_prueba_red +"model_selfsupervised_filter_mnist.h5")
print("modelo cargado")
 

modelo cargado


In [0]:
# Carga de data
from keras.datasets import cifar10, mnist
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
(x_train, y_train), (x_test, y_test) = mnist.load_data()


15000

In [0]:
from keras import backend as K

inp = model_filter.input                                           
outputs = [layer.output for layer in model_filter.layers[-2:]]         
functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    

layer_outs = [[func([x_train[i].reshape(1,28,28,1), 1.]) for func in functors] for i in range(len(x_train))]
len(layer_outs)

60000

In [0]:
features_filter = [layer_outs[i][0][0][0] for i in range(len(layer_outs))]

In [0]:
import numpy as np
features_filter= np.array(features_filter)
features_filter= features_filter.reshape(len(x_train),1,512)
x_train = x_train[:]
y_train = y_train[:]

In [0]:
from keras.utils import to_categorical

trainY_array = np.array(y_train)
y_train = to_categorical(trainY_array,num_classes=None, dtype='float32')

In [0]:
x_train = x_train.reshape(x_train.shape[0],28,28,1)

In [0]:
modelo_resnet.fit([x_train,features_filter],y_train,epochs=100,batch_size=1000,validation_data=[[x_train,features_filter],y_train])

Train on 60000 samples, validate on 60000 samples
Epoch 1/100
60000/60000 [==============================] - 20s 326us/step - loss: 0.8551 - acc: 0.9440 - val_loss: 0.6169 - val_acc: 0.9764
Epoch 2/100
60000/60000 [==============================] - 10s 173us/step - loss: 0.5300 - acc: 0.9887 - val_loss: 0.4834 - val_acc: 0.9887
Epoch 3/100
60000/60000 [==============================] - 11s 176us/step - loss: 0.4349 - acc: 0.9929 - val_loss: 0.4194 - val_acc: 0.9864
Epoch 4/100
60000/60000 [==============================] - 11s 179us/step - loss: 0.3716 - acc: 0.9934 - val_loss: 0.3858 - val_acc: 0.9798
Epoch 5/100
60000/60000 [==============================] - 11s 179us/step - loss: 0.3210 - acc: 0.9942 - val_loss: 0.3180 - val_acc: 0.9881
Epoch 6/100
60000/60000 [==============================] - 11s 178us/step - loss: 0.2785 - acc: 0.9953 - val_loss: 0.2819 - val_acc: 0.9884
Epoch 7/100
60000/60000 [==============================] - 11s 175us/step - loss: 0.2453 - acc: 0.9959 - val_l

In [0]:
# serialize model to JSON
model_json = modelo_resnet.to_json()
with open(carpeta_drive_prueba_red + "model_selfsupervised_filter_conjunto_mnist.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
modelo_resnet.save_weights(carpeta_drive_prueba_red + "model_selfsupervised_filter_conjunto_mnist.h5")
print("Saved model to disk")

Saved model to disk


# Co training con 2 vistas

In [0]:
#Cargando modelo de filtros-feature
from keras.models import model_from_json

json_file = open(carpeta_drive_prueba_red + 'model_selfsupervised_filter.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_filter_feature = model_from_json(loaded_model_json)
# load weights into new model
model_filter_feature.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_filter_feature.load_weights(carpeta_drive_prueba_red +"model_selfsupervised_filter.h5")
print("modelo cargado")
















modelo cargado


In [0]:
#Cargando modelo de rotacion-feature
json_file = open(carpeta_drive_prueba_red + 'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_rotation_feature = model_from_json(loaded_model_json)
# load weights into new model
model_rotation_feature.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_rotation_feature.load_weights(carpeta_drive_prueba_red +"model.h5")
print("modelo cargado")


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
modelo cargado


In [0]:
#Cargar modelo de rotacion-conjunto

json_file = open(carpeta_drive_prueba_red + 'model_selfsupervised.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_rotation_conjunto = model_from_json(loaded_model_json)
# load weights into new model
model_rotation_conjunto.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_rotation_conjunto.load_weights(carpeta_drive_prueba_red +"model_selfsupervised.h5")
print("modelo cargado")

modelo cargado


In [0]:
#Cargar modelo de filtros-conjunto
json_file = open(carpeta_drive_prueba_red + 'model_selfsupervised_filter_conjunto.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_filter_conjunto = model_from_json(loaded_model_json)
# load weights into new model
model_filter_conjunto.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_filter_conjunto.load_weights(carpeta_drive_prueba_red +"model_selfsupervised_filter_conjunto.h5")
print("modelo cargado")

modelo cargado


In [0]:
from keras import backend as K
def layers_extr(model,dat,name_model = "filter"):
  if(name_model == "filter"):
    inp = model.input                                           
    outputs = [layer.output for layer in model.layers[-2:]]         
    functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    

    layer_outs = [[func([dat[i].reshape(1,32,32,3), 1.]) for func in functors] for i in range(len(dat))]
    features = [layer_outs[i][0][0][0] for i in range(len(layer_outs))]
  elif(name_model == "rotation"):
    inp = model.input                                           
    outputs = [layer.output for layer in model.layers[1:]]         
    functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    

    layer_outs = [[func([dat[i].reshape(1,32,32,3), 1.]) for func in functors] for i in range(len(dat))]
    features = [layer_outs[i][5][0][0] for i in range(len(layer_outs))]
  return features


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
def extract_data_cifar(n_per_class,data_x,data_y):
    data_x_aux = []
    data_y_aux = []

    for i in range(10):
        aux = np.where(data_y == i)
        for cont in range(n_per_class):
            
            data_x_aux.append(data_x[aux[0][cont]])
            data_y_aux.append(data_y[aux[0][cont]])

            
    return(data_x_aux, data_y_aux)
import random
def des_ordenar_par(a,b):
    aux1 = []
    aux2 = []

    while len(a) > 0:
        num = random.randint(0,len(a)-1)
        aux1.append(a[num])
        aux2.append(b[num])

        del(a[num])
        del(b[num])
    return(aux1,aux2)

In [0]:
datos_n = X_train[-2000:-1]
datos_e, train_y = extract_data_cifar(100,X_train,y_train)

In [0]:
datos_e,train_y = des_ordenar_par(datos_e,train_y)

In [0]:
datos_e,train_y = np.array(datos_e),np.array(train_y)

In [0]:
view_1 = model_rotation_conjunto
feature_1 = model_rotation_feature
view_2 = model_filter_conjunto
feature_2 = model_filter_feature

In [0]:
filter1 = layers_extr(feature_1,datos_e,"rotation")
filter2 = layers_extr(feature_2,datos_e,"filter")


In [0]:
filter1 = np.array(filter1)
filter2 = np.array(filter2)

In [0]:
filter1= filter1.reshape(filter1.shape[0],1,filter1.shape[1])
filter2= filter2.reshape(filter2.shape[0],1,filter2.shape[1])

In [0]:
from keras.utils import to_categorical

# aux_y_train = np.array(y_train)
train_y = to_categorical(train_y,num_classes=None, dtype='float32')

In [0]:
print(train_y[:10])
print(train_y[-10:])

In [0]:
view_1.fit([datos_e,filter1],train_y,epochs=50,batch_size=50)

In [0]:
view_2.fit([datos_e,filter2],train_y,epochs=50,batch_size=50)

In [0]:
filter1 = layers_extr(feature_1,datos_n,"rotation")
filter2 = layers_extr(feature_2,datos_n,"filter")

In [0]:
filter1 = np.array(filter1)
filter2 = np.array(filter2)

In [0]:
filter1= filter1.reshape(filter1.shape[0],1,filter1.shape[1])
filter2= filter2.reshape(filter2.shape[0],1,filter2.shape[1])

In [0]:
result_view1 = view_1.predict([datos_n,filter1])
result_view2 = view_2.predict([datos_n,filter2])

In [0]:
def coincidencia(view_1,view_2):
  datos_correlacion = []
  cont = 0
  while cont < len(view_1):
    if(np.argmax(view_1[cont]) == np.argmax(view_2[cont]) and max(view_1[cont]) > 0.50 and max(view_2[cont]) > 0.50):
      aux = [0 if i != np.argmax(view_1[cont]) else 1 for i in range(len(view_1[cont]))]
      datos_correlacion.append([cont,np.array(np.argmax(aux))])
    cont+=1
  return datos_correlacion


In [0]:
datos_u = coincidencia(result_view1,result_view2)
print(len(datos_u))

741


In [0]:
datos_n_aux = datos_n.tolist()

In [0]:
def parse_datos_ytrain(datos_correlacion,y_train):
  train_aux = y_train.tolist()
  for i in datos_correlacion:
    train_aux.append([i[1].tolist()])
  return train_aux
def parse_datos_xtrain(datos_correlacion,X_train,datos_n):
  train_aux = X_train.tolist()
  datos_n_aux = datos_n.tolist()
  for i in datos_correlacion:
    train_aux.append(datos_n_aux[i[0]])
  cont_del = 0
  for i in datos_correlacion:
    del(datos_n_aux[i[0] - cont_del])
    cont_del+=1
  return train_aux,datos_n_aux

In [0]:
aux_y_train = parse_datos_ytrain(datos_u,train_y)
aux_y_train = np.array(aux_y_train)

In [0]:
print(aux_y_train[:10])
print(aux_y_train[-10:])


[list([0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0])
 list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0])
 list([0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0])
 list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0])
 list([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])
 list([0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])
 list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])]
[list([3]) list([3]) list([5]) list([8]) list([7]) list([0]) list([2])
 list([8]) list([2]) list([9])]


In [0]:
datos_e_aux,datos_n_aux = parse_datos_xtrain(datos_u,datos_e,datos_n)
datos_e_aux = np.array(datos_e_aux)
datos_n_aux = np.array(datos_n_aux)

In [0]:
filter1_train = layers_extr(feature_1,datos_e_aux,"rotation")
filter2_train = layers_extr(feature_2,datos_e_aux,"filter")

In [0]:
filter1_train = np.array(filter1_train)
filter2_train = np.array(filter2_train)

In [0]:
filter1_train= filter1_train.reshape(filter1_train.shape[0],1,filter1_train.shape[1])
filter2_train= filter2_train.reshape(filter2_train.shape[0],1,filter2_train.shape[1])

In [0]:
def max_y_train(aux_y_train):
  aux_np_y_train = []
  for i in aux_y_train:
    if(len(i)>2):
      aux_np_y_train.append([(np.argmax(i))])
    else:
      aux_np_y_train.append(i)

  aux_np_y_train = np.array(aux_np_y_train)
  return aux_np_y_train

In [0]:
# aux_y_train = np.array(y_train)
aux_y_train_2 = max_y_train(aux_y_train)
aux_y_train_2 = to_categorical(aux_y_train_2,num_classes=None, dtype='float32')


In [0]:
print(prueba[:10])
print(prueba[-10:])


[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [0]:
view_1.fit([datos_e_aux,filter1_train],aux_y_train_2,epochs=70,batch_size=100)

In [0]:
view_2.fit([datos_e_aux,filter2_train],aux_y_train_2,epochs=70,batch_size=100)

In [0]:
result_view1_2 = view_1.predict([datos_n_aux,filter1_train])
result_view2_2 = view_2.predict([datos_n_aux,filter2_train])

In [0]:
datos_u_2 = coincidencia(result_view1_2,result_view2_2)
print(len(datos_u_2))

353


In [0]:
aux_y_train_2 = max_y_train(aux_y_train_2)

In [0]:
aux_y_train_3 = parse_datos_ytrain(datos_u_2,aux_y_train_2)
aux_y_train_3 = np.array(aux_y_train_3)

In [0]:
datos_e_aux_2,datos_n_aux_2 = parse_datos_xtrain(datos_u_2,datos_e_aux,datos_n_aux)
datos_e_aux_2 = np.array(datos_e_aux_2)
datos_n_aux_2 = np.array(datos_n_aux_2)

In [0]:
filter1_train = layers_extr(feature_1,datos_e_aux_2,"rotation")
filter2_train = layers_extr(feature_2,datos_e_aux_2,"filter")

In [0]:
filter1_train = np.array(filter1_train)
filter2_train = np.array(filter2_train)

In [0]:
filter1_train= filter1_train.reshape(filter1_train.shape[0],1,filter1_train.shape[1])
filter2_train= filter2_train.reshape(filter2_train.shape[0],1,filter2_train.shape[1])

In [0]:
aux_y_train_3 = to_categorical(aux_y_train_3,num_classes=None, dtype='float32')

In [0]:
print(aux_y_train_3[:10])
print(aux_y_train_3[-10:])


[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [0]:
view_1.fit([datos_e_aux_2,filter1_train],aux_y_train_3,epochs=50,batch_size=100)

Epoch 1/50
2094/2094 [==============================] - 1s 454us/step - loss: 0.5246 - acc: 0.9097
Epoch 2/50
2094/2094 [==============================] - 1s 342us/step - loss: 0.4217 - acc: 0.9408
Epoch 3/50
2094/2094 [==============================] - 1s 345us/step - loss: 0.2945 - acc: 0.9814
Epoch 4/50
2094/2094 [==============================] - 1s 347us/step - loss: 0.2349 - acc: 0.9919
Epoch 5/50
2094/2094 [==============================] - 1s 346us/step - loss: 0.2226 - acc: 0.9967
Epoch 6/50
2094/2094 [==============================] - 1s 349us/step - loss: 0.2194 - acc: 0.9952
Epoch 7/50
2094/2094 [==============================] - 1s 343us/step - loss: 0.1996 - acc: 0.9981
Epoch 8/50
2094/2094 [==============================] - 1s 348us/step - loss: 0.2026 - acc: 0.9976
Epoch 9/50
2094/2094 [==============================] - 1s 342us/step - loss: 0.1931 - acc: 0.9990
Epoch 10/50
2094/2094 [==============================] - 1s 345us/step - loss: 0.1904 - acc: 0.9995
Epoch 11/

In [0]:
view_2.fit([datos_e_aux_2,filter2_train],aux_y_train_3,epochs=50,batch_size=100)

Epoch 1/50
2094/2094 [==============================] - 1s 357us/step - loss: 0.6143 - acc: 0.8763
Epoch 2/50
2094/2094 [==============================] - 1s 362us/step - loss: 0.4282 - acc: 0.9379
Epoch 3/50
2094/2094 [==============================] - 1s 344us/step - loss: 0.3117 - acc: 0.9733
Epoch 4/50
2094/2094 [==============================] - 1s 350us/step - loss: 0.2751 - acc: 0.9809
Epoch 5/50
2094/2094 [==============================] - 1s 349us/step - loss: 0.2816 - acc: 0.9766
Epoch 6/50
2094/2094 [==============================] - 1s 343us/step - loss: 0.2961 - acc: 0.9761
Epoch 7/50
2094/2094 [==============================] - 1s 352us/step - loss: 0.3046 - acc: 0.9699
Epoch 8/50
2094/2094 [==============================] - 1s 344us/step - loss: 0.2824 - acc: 0.9823
Epoch 9/50
2094/2094 [==============================] - 1s 345us/step - loss: 0.2370 - acc: 0.9924
Epoch 10/50
2094/2094 [==============================] - 1s 342us/step - loss: 0.2260 - acc: 0.9967
Epoch 11/

In [0]:
result_view1_3 = view_1.predict([datos_e_aux_2,filter1_train])
result_view2_3 = view_2.predict([datos_e_aux_2,filter2_train])

In [0]:
datos_u_3 = coincidencia(result_view1_3,result_view2_3)
print(len(datos_u_3))

2093


In [0]:
len(datos_e_aux_2)

2094

In [0]:
aux_y_train_4 = max_y_train(aux_y_train_3)

In [0]:
aux_y_train_4 = parse_datos_ytrain(datos_u_3,aux_y_train_4)
aux_y_train_4 = np.array(aux_y_train_4)

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
test_real = y_train[-2000:-1]
test_real_x = X_train[-2000:-1]
# print(np.where(len((datos_e_aux_2 == datos_e_aux_2[100])[0]) == 3072))


# print(len(test_real_x[0]),len(test_real_x[0][0]),len(test_real_x[0][0][0]))

In [0]:
def buscar_y(bus,a):
  lista = []
  for i in range(len(bus)):
    ac = np.where(a==bus[i])
    for x in range(len(a)):
      cu = np.where(ac[0] == x)
      if(len(cu[0]) >= 3072):
        lista.append(x)
  return lista

In [0]:
pos = buscar_y(datos_e_aux_2,test_real_x)

In [0]:
real[0]

array([9], dtype=uint8)

In [0]:
real = []
for i in pos:
  real.append(y_train[i])


In [0]:
datos_pred = []
for i in result_view1_3:
  datos_pred.append(np.argmax(i))
  

In [0]:
test_pred = datos_pred[:1000]
# real =  real
print(len(real),len(test_pred))

1094 1000


In [0]:
test_pred = max_y_train(test_pred)
# real = max_y_train(real)

In [0]:
cont = 0
for i in range(len(test_pred)):
  if(y_train[i][0] == test_pred[i]):
    cont+=1
  # else:
  #   # print(i)
  #   print(real[i],test_pred[i])
  #   print("--")
print(cont/len(test_pred))
print(y_train[0][0],test_pred[0].tolist())
len(test_pred)

0.09
6 5


1000

In [0]:
# serialize model to JSON
model_json = view_1.to_json()
with open("model_final_view1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
view_1.save_weights("model_final_view1.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
# serialize model to JSON
model_json = view_2.to_json()
with open("model_final_view2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
view_2.save_weights("model_final_view2.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
spec = importlib.util.spec_from_file_location("modelo_resnet3", "drive/My Drive/Colab Notebooks/Prueba red/modelo_resnet3.py")
archivo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(archivo)


model1 = archivo.ResnetBuilder.build_resnet_18((3,32, 32), 10)
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model2 = archivo.ResnetBuilder.build_resnet_18((3,32, 32), 10)
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

dont tf
dont tf


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [0]:
train_X = X_train[:1000]
train_y = y_train[:1000]
test_X = X_train[1000:3000]

In [0]:
train_y = to_categorical(train_y,num_classes=None, dtype='float32')

In [0]:
model1.fit(train_X,train_y,epochs=50,batch_size=100)
model2.fit(train_X,train_y,epochs=50,batch_size=100)

Epoch 1/50
1000/1000 [==============================] - 7s 7ms/step - loss: 3.0141 - acc: 0.2770
Epoch 2/50
1000/1000 [==============================] - 0s 330us/step - loss: 1.9292 - acc: 0.6100
Epoch 3/50
1000/1000 [==============================] - 0s 325us/step - loss: 1.2446 - acc: 0.8480
Epoch 4/50
1000/1000 [==============================] - 0s 324us/step - loss: 0.8173 - acc: 0.9800
Epoch 5/50
1000/1000 [==============================] - 0s 329us/step - loss: 0.6979 - acc: 0.9920
Epoch 6/50
1000/1000 [==============================] - 0s 330us/step - loss: 0.6471 - acc: 0.9980
Epoch 7/50
1000/1000 [==============================] - 0s 323us/step - loss: 0.6306 - acc: 0.9990
Epoch 8/50
1000/1000 [==============================] - 0s 326us/step - loss: 0.6202 - acc: 0.9990
Epoch 9/50
1000/1000 [==============================] - 0s 343us/step - loss: 0.6069 - acc: 1.0000
Epoch 10/50
1000/1000 [==============================] - 0s 327us/step - loss: 0.5979 - acc: 1.0000
Epoch 11/50

In [0]:
result_view1 = model1.predict(test_X)
result_view2 = model2.predict(test_X)

In [0]:
datos_u = coincidencia(result_view1,result_view2)
print(len(datos_u))

730


In [0]:
cont = 0
y_train_test = y_train[1000:3000]
for i in range(len(result_view2)):
  if(y_train_test[i][0] == np.argmax(result_view2[i])):
    cont+=1
print(cont/len(result_view1))

0.383
